# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [11]:
import os

from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [12]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key = OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages = context,
            temperature = 0,
        )

    return (response.choices[0].message.content)

In [37]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['first_name', 'last_name', 'health_insurance'], # ENTER A TABLE COLUMNS HERE,
        'definition': ['The first name of the client', 'The last name of the client', 'The health insurance of the client'] # ENTER A TABLE DEFINITATIONS HERE
}

df = pd.DataFrame(data)
print(df)

              table                          definition
0        first_name        The first name of the client
1         last_name         The last name of the client
2  health_insurance  The health insurance of the client


In [40]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [41]:
print(text_tables)

first_name: The first name of the client
last_name: The last name of the client
health_insurance: The health insurance of the client


In [25]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Question:
{question}
"""


In [26]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables = text_tables, question = 'Where can I change the last name of the client') # ENTER YOUR QUERY HERE)

In [22]:
print(return_OAI(pqt1))

{
    "tables": ["last_name"]
}


In [27]:
pqt2 = prompt_question_tables.format(tables = text_tables,
                                     question = 'Where can I change the days left of the membership of a client') # ENTER YOUR QUERY HERE)

In [ ]:
print(return_OAI(pqt2))

{
    "tables": ["client_membership"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [32]:
pqt3 = prompt_question_tables.format(tables = text_tables,
                                     question = 'Where can I store the client\'s credit')

print(return_OAI(pqt3))

{
    "tables": ["client"]
}


In [34]:
pqt4 = prompt_question_tables.format(tables = text_tables,
                                     question = 'Where can I use the client\'s debt')

print(return_OAI(pqt4))

{
    "tables": ["debt"]
}


In [42]:
pqt5 = prompt_question_tables.format(tables = text_tables,
                                     question = 'Where can I store the client\'s insurance')

print(return_OAI(pqt5))

```json
{
    "tables": ["health_insurance"]
}
```


In [43]:
# Report Writing
print("""Sometimes it creates a contextual answer to the thing I ask it and sometimes it creates something far off. Especially if it does not have something related in the databse.""")

Sometimes it creates a contextual answer to the thing I ask it and sometimes it creates something far off. Especially if it does not have something related in the databse.
